# Laboratório 05: Expansão de Consultas

Antes de responder as perguntas, precisamos dos documentos que serão extraídos do CSV, além das 5 consultas de um termo escolhidas na questão 2.1 do Laboratório 04 e da lista invertida obtida anteriormente.



*  **Importação das extensões necessárias** 

In [10]:
import csv
import pandas as pd
import nltk
import re
import collections
import math
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('rslp')
import heapq
import time
import bisect

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!



*   **Leitura do CSV**

In [0]:
ranking = pd.read_csv('https://raw.githubusercontent.com/LDVictor/ri_lab_05/master/dados.csv')



*  **Definição das 5 consultas**

In [0]:
consultas = ["presidente", "educação", "privatização", "empresa", "bolsonaro"]

# Questão 1

**Calcule as top-10 palavras mais associadas a cada uma dessas 5 palavras de acordo com as 4 métricas que vimos na aula. Você deve produzir uma tabela similar à tabela 6.3 do capítulo 6 do livro texto (pág. 204).**


Primeiramente, iremos definir as funções para calcular cada uma das métricas: Dice's Coefficient (Dice), Mutual Information Measure (MIM), Expected Mutual Information Measure (EMIM) e Pearson's Chi-squared (χ²).

In [0]:
def valorDice(Nab, Na, Nb, palavra):
  return (2*Nab / (Na + Nb), str(palavra[0]))

def valorMIM(Nab, Na, Nb, N, palavra):
  return (math.log(N, 10) * (Nab / (Na * Nb)), str(palavra[0]))

def valorEMIM(Nab, Na, Nb, N, palavra):
  return (Nab * math.log(N * ((Nab) / Na*Nb), 10), str(palavra[0]))

def valorChiSquared(Nab, Na, Nb, N, palavra):
  return ((Nab - (N * (Na / N) * (Nb / N)) ** 2) / (N * (Na/N) * (Nb/N)), str(palavra[0]))

De maneira análoga, também precisamos definir como calcular os valores de Na, Nb, Nab e N para usar nas métricas definidas.

In [0]:
def calculaNx(palavraX):
  for i in range(len(ranking)-1):
    if (palavraX == ranking[0][i]):
      return len(ranking[1][i])
  return 0;
    
def calculaNab(palavraA, palavraB):
  for i in range(len(ranking)-1):
    if (palavraA == ranking[0][i]):
      for j in range(len(ranking)-1):
        if (palavraB == ranking[0][j]):
          return len(ranking[1][i]) + len(ranking[1][j])
  return 0;

def calculaN():
  return len(ranking)

Com as funções já criadas, podemos agora percorrer a lista invertida para cada consulta, armazenando os resultados dos cálculos de cada métrica. Para isso, criaremos uma função que, dada a consulta, retornará uma tabela com as 10 palavras mais associadas a cada consulta de acordo com as métricas que definimos acima.

In [0]:
def criaTabela(consulta):
  dice = []
  mim = []
  emim = []
  chi_squared = []
  
  for coluna in ranking.itertuples():
    palavra = [coluna[1], coluna[2]]
    if palavra[0] != consulta:
      bisect.insort(dice, valorDice(calculaNx(consulta), calculaNx(palavra[0]), calculaNab(consulta, palavra[0]), palavra))
      bisect.insort(mim, valorMIM(calculaNx(consulta), calculaNx(palavra[0]), calculaNab(consulta, palavra[0]), calculaN(), palavra))
      bisect.insort(emim, valorEMIM(calculaNx(consulta), calculaNx(palavra[0]), calculaNab(consulta, palavra[0]), calculaN(), palavra))
      bisect.insort(chi_squared, valorChiSquared(calculaNx(consulta), calculaNx(palavra[0]), calculaNab(consulta, palavra[0]), calculaN(), palavra))

  dice.reverse()    
  mim.reverse()
  emim.reverse()
  chi_squared.reverse()
  
  tabela_metricas = pd.DataFrame(columns=['MIM', 'EMIM', 'χ²', 'Dice'])

  tabela_metricas['MIM'] = [palavra[1] for palavra in mim]
  tabela_metricas['EMIM'] = [palavra[1] for palavra in emim]
  tabela_metricas['χ²'] = [palavra[1] for palavra in chi_squared]
  tabela_metricas['Dice'] = [palavra[1] for palavra in dice]
  
  tabela_metricas.index += 1
  
  return tabela_metricas.head(10)

Por fim, chamaremos a função para cada consulta do vetor.




1.   Consulta "presidente"

In [74]:
tabela_consulta_1 = criaTabela(consultas[0])
tabela_consulta_1

KeyError: ignored


2.   Consulta "educação"



In [75]:
tabela_consulta_2 = criaTabela(consultas[1])
tabela_consulta_2

KeyError: ignored


3.   Consulta "privatização"

In [0]:
tabela_consulta_3 = criaTabela(consultas[2])
tabela_consulta_3

,MIM,EMIM,χ²,Dice



4.   Consulta "empresa"

In [0]:
tabela_consulta_4 = criaTabela(consultas[3])
tabela_consulta_4

,MIM,EMIM,χ²,Dice




5.   Consulta "bolsonaro"


In [0]:
tabela_consulta_5 = criaTabela(consultas[4])
tabela_consulta_5

,MIM,EMIM,χ²,Dice



**Qual métrica você acha que obteve os melhores resultados? Por que?**

...

# Questão 2

**De acordo com a métrica que deu os melhores resultados na sua opinião, execute agora cada consulta (usando a abordagem documento- ou termo-por-vez)  expandido-a com: os top-3, top-5 e top-10 documentos.**

**O que acontece com a precisão dos resultados em cada caso? Aumenta ou diminui? Justifique bem sua resposta.**

...